In [1]:
import librosa
import numpy as np
import os
import glob
from tqdm import tqdm_notebook
import sklearn
import pandas as pd
import json # import json module
import librosa, soundfile

from PIL import Image  # Python Image Library 이미지 라이브러리
import matplotlib.pyplot as plt  # 이미지 보는 라이브러리

import sys
sys.path.append('../../')
from CommonUtil import *

In [7]:
data_dir = '/home/gil/gcubme_99/1. 연구 프로젝트/1. 인공지능/2022_NIA 46번 의료음성데이터/1. 데이터/구음장애'
print(os.path.isdir(data_dir))


True


In [83]:
CODE_SPLIT = 38
TOTAL_SPLIT = 40

WAVE_DIR = [f'{data_dir}/원천데이터/원천데이터'\
            , f'{data_dir}/원천데이터/sorizava']

JSON_DIR = [f'{data_dir}/라벨링데이터'\
            , f'{data_dir}/원천데이터/sorizava']

SAVE_DIR = './data/origin2'

SAMPLE_RATE = 16000

wave_dir = WAVE_DIR
json_dir = JSON_DIR
save_dir = SAVE_DIR

os.makedirs(save_dir, exist_ok=True)

print(wave_dir)
print(json_dir)
print(save_dir)


['/home/gil/gcubme_99/1. 연구 프로젝트/1. 인공지능/2022_NIA 46번 의료음성데이터/1. 데이터/구음장애/원천데이터/원천데이터', '/home/gil/gcubme_99/1. 연구 프로젝트/1. 인공지능/2022_NIA 46번 의료음성데이터/1. 데이터/구음장애/원천데이터/sorizava']
['/home/gil/gcubme_99/1. 연구 프로젝트/1. 인공지능/2022_NIA 46번 의료음성데이터/1. 데이터/구음장애/라벨링데이터', '/home/gil/gcubme_99/1. 연구 프로젝트/1. 인공지능/2022_NIA 46번 의료음성데이터/1. 데이터/구음장애/원천데이터/sorizava']
./data/origin2


# 웨이브 리스트 가져오기

In [13]:
wave_path_dir = []
for DIR in WAVE_DIR:
    wavs = sorted(glob.glob(f'{DIR}/**/*_SCO_*.wav', recursive=True))
    wave_path_dir.extend(wavs)
print(len(wave_path_dir))

2663420


In [14]:
json_path_dir = []
for DIR in JSON_DIR:
    jsons = sorted(glob.glob(f'{DIR}/**/*_SCO_*.json', recursive=True))
    json_path_dir.extend(jsons)
print(len(json_path_dir))

2663421


In [44]:
wave_list = wave_path_dir
json_list = json_path_dir

In [45]:
# wav_list = sorted(glob.glob(f'{data_dir}/**/*_SCO_*.wav', recursive=True))
# print(len(wav_list))

In [46]:
# json_list = sorted(glob.glob(f'{json_dir}/**/*_SCO_*.json', recursive=True))
# print(len(json_list))

# 환자 아이디 추출

In [47]:
patient_list = []
for a in tqdm_notebook(range(len(wave_list))):
    wave_path = wave_list[a]
    patient_id = wave_path.split('/')[-1].split('_')[0]
    
    if patient_id in patient_list:
        continue
    else:
        patient_list.append(patient_id)
print(len(patient_list))

/tmp/ipykernel_3422563/3795215625.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for a in tqdm_notebook(range(len(wave_list))):


  0%|          | 0/2663420 [00:00<?, ?it/s]

1002


In [49]:
patient_wave_list = []
patient_wave_cnt = []
for b in range(len(patient_list)):
    patient_wave_list.append([])
    patient_wave_cnt.append(0)

for a in tqdm_notebook(range(len(wave_list))):
    wave_path = wave_list[a]
    patient_id = wave_path.split('/')[-1].split('_')[0]
    
    for b in range(len(patient_list)):
        if patient_id == patient_list[b]:
            patient_wave_list[b].append(wave_path)
            patient_wave_cnt[b] += 1
            break
            

/tmp/ipykernel_3422563/562176992.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for a in tqdm_notebook(range(len(wave_list))):


  0%|          | 0/2663420 [00:00<?, ?it/s]

In [52]:
#0번째 환자의 1번째 데이터 path
print(patient_wave_list[0][1])

/home/gil/gcubme_99/1. 연구 프로젝트/1. 인공지능/2022_NIA 46번 의료음성데이터/1. 데이터/구음장애/원천데이터/원천데이터/1006_wav/1006_wav/01.뇌경색/SCO/01.IoT/HM0006_SCO_A_1_002.wav


In [50]:
print(len(patient_wave_cnt), np.min(patient_wave_cnt), np.max(patient_wave_cnt))
print(sorted(patient_wave_cnt))

1002 5 6016
[5, 5, 25, 47, 50, 59, 59, 59, 59, 59, 66, 69, 69, 69, 73, 74, 75, 75, 75, 75, 75, 75, 75, 75, 116, 118, 132, 135, 143, 143, 144, 144, 144, 144, 152, 177, 184, 197, 202, 203, 203, 203, 203, 203, 216, 219, 225, 236, 253, 276, 276, 277, 277, 277, 277, 315, 323, 323, 336, 354, 383, 408, 408, 433, 438, 456, 460, 476, 477, 526, 561, 567, 568, 568, 611, 621, 634, 634, 672, 675, 680, 692, 693, 693, 693, 693, 693, 699, 702, 709, 720, 722, 731, 741, 744, 745, 749, 750, 750, 751, 751, 751, 751, 751, 751, 752, 752, 752, 752, 752, 752, 752, 752, 752, 752, 752, 752, 752, 752, 752, 752, 752, 752, 752, 752, 752, 752, 752, 752, 752, 752, 752, 752, 752, 752, 752, 870, 886, 920, 950, 1027, 1036, 1059, 1134, 1159, 1191, 1225, 1234, 1249, 1254, 1310, 1328, 1479, 1498, 1502, 1503, 1503, 1503, 1503, 1503, 1503, 1504, 1504, 1504, 1504, 1504, 1504, 1504, 1504, 1504, 1504, 1505, 1533, 1545, 1558, 1562, 1579, 1582, 1584, 1604, 1619, 1635, 1669, 1722, 1757, 1858, 1868, 1909, 1981, 2061, 2077, 2200, 2

In [61]:
data_cnt = 25
patient_wave_list_s =[]

for a in tqdm_notebook(range(len(patient_wave_list))):
    w_list = patient_wave_list[a]
    if len(w_list) < data_cnt:
        continue
    w_list = shuffle(w_list)
    patient_wave_list_s.append(w_list[:data_cnt])
            

/tmp/ipykernel_3422563/3197375507.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for a in tqdm_notebook(range(len(patient_wave_list))):


  0%|          | 0/1002 [00:00<?, ?it/s]

In [62]:
print(len(patient_wave_list_s))
print(len(patient_wave_list_s)*data_cnt)


1000
25000


# 데이터 나누기

In [84]:
def split_data(X_data_y, train_rate:float=0.8, val_rate:float=0.9):
    X_data_y = np.array(X_data_y)
    print(X_data_y.shape)
    X_data_y = shuffle(X_data_y)
    
    
    h = X_data_y.shape[0]
    train_num = int(h * train_rate)
    val_num = int(h * val_rate)

    train_num = round(train_num,0)
    val_num = round(val_num,0)
    
    train = X_data_y[:train_num]
    val = X_data_y[train_num:val_num]
    test = X_data_y[val_num:]
    
    return train, val, test

In [85]:
train_wave_list, val_wave_list, test_wave_list = split_data(patient_wave_list_s)
print(len(train_wave_list))
print(len(val_wave_list))
print(len(test_wave_list))


(1000, 25)
800
100
100


In [86]:
def return_total_list(data_list):
    total_list = []
    for a in range(len(data_list)):
        data = data_list[a]
        total_list.extend(data)
    return total_list

train_list = return_total_list(train_wave_list)
val_list = return_total_list(val_wave_list)
test_list = return_total_list(test_wave_list)
    
print(len(train_list), len(val_list), len(test_list), '\t', len(train_list)+len(val_list)+len(test_list))

20000 2500 2500 	 25000


In [87]:
print(train_list[0])
print(os.path.basename(train_list[0]))
print(os.path.basename(train_list[0]).split('.wav')[0])
print(os.path.dirname(train_list[0]))


/home/gil/gcubme_99/1. 연구 프로젝트/1. 인공지능/2022_NIA 46번 의료음성데이터/1. 데이터/구음장애/원천데이터/원천데이터/1216_HC_1_wav/01.뇌경색/SCO/05.교통정보/HC0048_SCO_E_1_047.wav
HC0048_SCO_E_1_047.wav
HC0048_SCO_E_1_047
/home/gil/gcubme_99/1. 연구 프로젝트/1. 인공지능/2022_NIA 46번 의료음성데이터/1. 데이터/구음장애/원천데이터/원천데이터/1216_HC_1_wav/01.뇌경색/SCO/05.교통정보


In [88]:
# with statement

print(CODE_SPLIT)
print(TOTAL_SPLIT)

#for a in range(len(file_list)):
for b in range(3):
    if b == 0:
        file_list = train_list
        tvt = 'train'
    elif b == 1:
        file_list = val_list
        tvt = 'validation'
    elif b == 2:
        file_list = test_list
        tvt = 'test'
        
    save_file_dir = f'{save_dir}/{tvt}'
    
    print(len(file_list))
    
    for a in tqdm_notebook(range(len(file_list))):
        wav_path = file_list[a]
        filename = os.path.basename(wav_path).split('.wav')[0]
        patient_id = wav_path.split('/')[-1].split('_')[0]
        
        json_save = f'{save_file_dir}/{patient_id}/{filename}.json'
        wav_save = f'{save_file_dir}/{patient_id}/{filename}.wav'
        
        os.makedirs(os.path.dirname(wav_save), exist_ok=True)
        os.makedirs(os.path.dirname(json_save), exist_ok=True)
        
        if os.path.isfile(json_save) and os.path.isfile(wav_save):
            continue
        
        json_path = [s for s in json_list if filename in s][0] 
        
        with open(json_path, 'r') as json_file:
            json_data = json.load(json_file)
        with open(json_save,'w') as f:
            json.dump(json_data, f, indent="\t", ensure_ascii=False)
        
        audio, sr = librosa.load(wav_path, sr=SAMPLE_RATE)
        soundfile.write(wav_save, audio, sr, format='WAV')
        

38
40
20000


/tmp/ipykernel_3422563/3264086470.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for a in tqdm_notebook(range(len(file_list))):


  0%|          | 0/20000 [00:00<?, ?it/s]

2500


  0%|          | 0/2500 [00:00<?, ?it/s]

2500


  0%|          | 0/2500 [00:00<?, ?it/s]